In [1]:
main_dir = '/Users/simon/bsose_monthly/'
salt = main_dir + 'bsose_i106_2008to2012_monthly_Salt.nc'
theta = main_dir + 'bsose_i106_2008to2012_monthly_Theta.nc'
density = main_dir + 'density.nc'
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import xarray as xr
xr.set_options(keep_attrs=True)
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature
import matplotlib.path as mpath
import pyxpcm
from pyxpcm.models import pcm


def pcm_fit_and_post(time_i=42, K=4, maxvar=2):  
    # Define features to use
    # Instantiate the PCM
    
    max_depth = 2000
    z = np.arange(-300, -max_depth, -10.)
    features_pcm = {'THETA': z, 'SALT': z}
    features = {'THETA': 'THETA', 'SALT': 'SALT'}
    salt_nc = xr.open_dataset(salt).isel(time=slice(42,54))#time_i)#
    theta_nc = xr.open_dataset(theta).isel(time=slice(42,54))#time_i)
    big_nc = xr.merge([salt_nc, theta_nc])
    both_nc = big_nc.where(big_nc.coords['Depth'] > 
                           max_depth).drop(['iter', 'Depth', 
                                            'rA', 'drF', 'hFacC'])   
 
    attr_d = {}

    for coord in both_nc.coords:
        attr_d[coord] = both_nc.coords[coord].attrs
        
    lons_new = np.linspace(both_nc.XC.min(), both_nc.XC.max(), 60*4)
    lats_new = np.linspace(both_nc.YC.min(), both_nc.YC.max(), 60)
    # ds = both_nc # .copy(deep=True)
    ds = both_nc.interp(coords={'YC': lats_new, 
                                'XC': lons_new})#, method='cubic')
        
    m = pcm(K=K, 
            features=features_pcm, 
            maxvar=maxvar, 
            timeit=True, 
            timeit_verb=1)
    m.fit(ds, features=features, dim='Z') #, inplace=True)
    m.predict(ds, features=features, 
              dim='Z', inplace=True)
    m.predict_proba(ds, features=features, 
                    dim='Z', inplace=True)
    m.find_i_metric(ds, inplace=True)
    
    def sanitize():
        del ds.PCM_LABELS.attrs['_pyXpcm_cleanable']
        del ds.PCM_POST.attrs['_pyXpcm_cleanable']
        del ds.PCM_RANK.attrs['_pyXpcm_cleanable']
        
    for coord in attr_d:
        ds.coords[coord].attrs = attr_d[coord]
    
    sanitize()
    return ds, m


/Users/simon/pyxpcm/pyxpcm/plot.py:45: UserWarning: pyXpcm requires seaborn installed for full plotting functionality
  warnings.warn("pyXpcm requires seaborn installed for full plotting functionality")


In [3]:
ds, m = pcm_fit_and_post(K=5)


  fit.1-preprocess.1-mask: 197 ms
[-2.100e+00 -6.700e+00 -1.215e+01 -1.855e+01 -2.625e+01 -3.525e+01
 -4.500e+01 -5.500e+01 -6.500e+01 -7.500e+01 -8.500e+01 -9.500e+01
 -1.050e+02 -1.150e+02 -1.250e+02 -1.350e+02 -1.465e+02 -1.615e+02
 -1.800e+02 -2.000e+02 -2.200e+02 -2.400e+02 -2.600e+02 -2.800e+02
 -3.010e+02 -3.270e+02 -3.610e+02 -4.025e+02 -4.500e+02 -5.000e+02
 -5.515e+02 -6.140e+02 -7.000e+02 -8.000e+02 -9.000e+02 -1.000e+03
 -1.100e+03 -1.225e+03 -1.400e+03 -1.600e+03 -1.800e+03 -2.010e+03
 -2.270e+03 -2.610e+03 -3.000e+03 -3.400e+03 -3.800e+03 -4.200e+03
 -4.600e+03 -5.000e+03 -5.400e+03 -5.800e+03]
  fit.1-preprocess.2-feature_THETA.1-ravel: 332 ms
[-2.100e+00 -6.700e+00 -1.215e+01 -1.855e+01 -2.625e+01 -3.525e+01
 -4.500e+01 -5.500e+01 -6.500e+01 -7.500e+01 -8.500e+01 -9.500e+01
 -1.050e+02 -1.150e+02 -1.250e+02 -1.350e+02 -1.465e+02 -1.615e+02
 -1.800e+02 -2.000e+02 -2.200e+02 -2.400e+02 -2.600e+02 -2.800e+02
 -3.010e+02 -3.270e+02 -3.610e+02 -4.025e+02 -4.500e+02 -5.000e+0

  predict_proba.1-preprocess.2-feature_THETA.4-scale_transform: 951 ms
  predict_proba.1-preprocess.2-feature_THETA.5-reduce_fit: 0 ms
  predict_proba.1-preprocess.2-feature_THETA.6-reduce_transform: 139 ms
  predict_proba.1-preprocess.2-feature_THETA.total: 1404 ms
  predict_proba.1-preprocess: 1404 ms
  predict_proba.1-preprocess.3-homogeniser: 3 ms
[-2.100e+00 -6.700e+00 -1.215e+01 -1.855e+01 -2.625e+01 -3.525e+01
 -4.500e+01 -5.500e+01 -6.500e+01 -7.500e+01 -8.500e+01 -9.500e+01
 -1.050e+02 -1.150e+02 -1.250e+02 -1.350e+02 -1.465e+02 -1.615e+02
 -1.800e+02 -2.000e+02 -2.200e+02 -2.400e+02 -2.600e+02 -2.800e+02
 -3.010e+02 -3.270e+02 -3.610e+02 -4.025e+02 -4.500e+02 -5.000e+02
 -5.515e+02 -6.140e+02 -7.000e+02 -8.000e+02 -9.000e+02 -1.000e+03
 -1.100e+03 -1.225e+03 -1.400e+03 -1.600e+03 -1.800e+03 -2.010e+03
 -2.270e+03 -2.610e+03 -3.000e+03 -3.400e+03 -3.800e+03 -4.200e+03
 -4.600e+03 -5.000e+03 -5.400e+03 -5.800e+03]
  predict_proba.1-preprocess.2-feature_SALT.1-ravel: 288 ms
[-2.

In [17]:
ds

<xarray.Dataset>
Dimensions:     (XC: 240, YC: 60, Z: 52, pcm_class: 5, time: 12)
Coordinates:
  * time        (time) datetime64[ns] 2011-08-01T15:12:00 ... 2012-07-01T09:36:00
  * Z           (Z) float32 -2.1 -6.7 -12.15 -18.55 ... -5000.0 -5400.0 -5800.0
  * YC          (YC) float64 -77.98 -77.16 -76.35 ... -31.35 -30.53 -29.72
  * XC          (XC) float64 0.08333 1.589 3.094 4.6 ... 355.4 356.9 358.4 359.9
Dimensions without coordinates: pcm_class
Data variables:
    SALT        (time, Z, YC, XC) float64 nan nan nan nan ... 0.0 0.0 0.0 0.0
    THETA       (time, Z, YC, XC) float64 nan nan nan nan ... 0.0 0.0 0.0 0.0
    PCM_LABELS  (time, YC, XC) float64 nan nan nan nan nan ... 4.0 4.0 4.0 4.0
    PCM_POST    (pcm_class, time, YC, XC) float64 nan nan nan ... 1.0 1.0 1.0
    PCM_RANK    (pcm_class, time, YC, XC) float64 nan nan nan ... 2.0 2.0 2.0

In [18]:
ds.to_netcdf('nc/trained_on_year.nc')

In [9]:
def label_and_save(m, time_i):
    # Define features to use
    # Instantiate the PCM
    
    max_depth = 2000
    z = np.arange(-300, -max_depth, -10.)
    features_pcm = {'THETA': z, 'SALT': z}
    features = {'THETA': 'THETA', 'SALT': 'SALT'}
    salt_nc = xr.open_dataset(salt).isel(time=time_i)
    theta_nc = xr.open_dataset(theta).isel(time=time_i)
    big_nc = xr.merge([salt_nc, theta_nc])
    both_nc = big_nc.where(big_nc.coords['Depth'] > 
                           max_depth).drop(['iter', 'Depth', 
                                            'rA', 'drF', 'hFacC'])   
 
    attr_d = {}

    for coord in both_nc.coords:
        attr_d[coord] = both_nc.coords[coord].attrs
    
    ds = both_nc
    
    m.predict(ds, features=features, 
              dim='Z', inplace=True)
    m.predict_proba(ds, features=features, 
                    dim='Z', inplace=True)
    
    def sanitize():
        del ds.PCM_LABELS.attrs['_pyXpcm_cleanable']
        del ds.PCM_POST.attrs['_pyXpcm_cleanable']
        
    for coord in attr_d:
        ds.coords[coord].attrs = attr_d[coord]
    
    sanitize()
    ds = ds.drop(['SALT', 'THETA']).astype('float32')
    
    ds = ds.expand_dims(dim='time', axis=None)

    ds = ds.assign_coords({"time":
            ("time", [salt_nc.coords['time'].values])})

    ds.coords['time'].attrs = salt_nc.coords['time'].attrs
    ds.to_netcdf('nc/labels/'+str(time_i)+'.nc')


In [ ]:
for time_i in range(0, 60):
    label_and_save(m, time_i)

  predict.1-preprocess.1-mask: 177 ms
[-2.100e+00 -6.700e+00 -1.215e+01 -1.855e+01 -2.625e+01 -3.525e+01
 -4.500e+01 -5.500e+01 -6.500e+01 -7.500e+01 -8.500e+01 -9.500e+01
 -1.050e+02 -1.150e+02 -1.250e+02 -1.350e+02 -1.465e+02 -1.615e+02
 -1.800e+02 -2.000e+02 -2.200e+02 -2.400e+02 -2.600e+02 -2.800e+02
 -3.010e+02 -3.270e+02 -3.610e+02 -4.025e+02 -4.500e+02 -5.000e+02
 -5.515e+02 -6.140e+02 -7.000e+02 -8.000e+02 -9.000e+02 -1.000e+03
 -1.100e+03 -1.225e+03 -1.400e+03 -1.600e+03 -1.800e+03 -2.010e+03
 -2.270e+03 -2.610e+03 -3.000e+03 -3.400e+03 -3.800e+03 -4.200e+03
 -4.600e+03 -5.000e+03 -5.400e+03 -5.800e+03]
  predict.1-preprocess.2-feature_THETA.1-ravel: 658 ms
[-2.100e+00 -6.700e+00 -1.215e+01 -1.855e+01 -2.625e+01 -3.525e+01
 -4.500e+01 -5.500e+01 -6.500e+01 -7.500e+01 -8.500e+01 -9.500e+01
 -1.050e+02 -1.150e+02 -1.250e+02 -1.350e+02 -1.465e+02 -1.615e+02
 -1.800e+02 -2.000e+02 -2.200e+02 -2.400e+02 -2.600e+02 -2.800e+02
 -3.010e+02 -3.270e+02 -3.610e+02 -4.025e+02 -4.500e+02 -

  predict.1-preprocess.2-feature_THETA.4-scale_transform: 6860 ms
  predict.1-preprocess.2-feature_THETA.5-reduce_fit: 0 ms
  predict.1-preprocess.2-feature_THETA.6-reduce_transform: 967 ms
  predict.1-preprocess.2-feature_THETA.total: 8605 ms
  predict.1-preprocess: 8605 ms
  predict.1-preprocess.3-homogeniser: 13 ms
[-2.100e+00 -6.700e+00 -1.215e+01 -1.855e+01 -2.625e+01 -3.525e+01
 -4.500e+01 -5.500e+01 -6.500e+01 -7.500e+01 -8.500e+01 -9.500e+01
 -1.050e+02 -1.150e+02 -1.250e+02 -1.350e+02 -1.465e+02 -1.615e+02
 -1.800e+02 -2.000e+02 -2.200e+02 -2.400e+02 -2.600e+02 -2.800e+02
 -3.010e+02 -3.270e+02 -3.610e+02 -4.025e+02 -4.500e+02 -5.000e+02
 -5.515e+02 -6.140e+02 -7.000e+02 -8.000e+02 -9.000e+02 -1.000e+03
 -1.100e+03 -1.225e+03 -1.400e+03 -1.600e+03 -1.800e+03 -2.010e+03
 -2.270e+03 -2.610e+03 -3.000e+03 -3.400e+03 -3.800e+03 -4.200e+03
 -4.600e+03 -5.000e+03 -5.400e+03 -5.800e+03]
  predict.1-preprocess.2-feature_SALT.1-ravel: 1331 ms
[-2.100e+00 -6.700e+00 -1.215e+01 -1.855e+0

  predict.1-preprocess.2-feature_SALT.4-scale_transform: 4461 ms
  predict.1-preprocess.2-feature_SALT.5-reduce_fit: 0 ms
  predict.1-preprocess.2-feature_SALT.6-reduce_transform: 918 ms
  predict.1-preprocess.2-feature_SALT.total: 6621 ms
  predict.1-preprocess: 6621 ms
  predict.1-preprocess.3-homogeniser: 13 ms
  predict.1-preprocess.4-xarray: 97 ms
  predict.1-preprocess: 17113 ms
  predict.predict: 257 ms
  predict.score: 298 ms
  predict.xarray: 3400 ms
  predict: 21073 ms
  predict_proba.1-preprocess.1-mask: 388 ms
[-2.100e+00 -6.700e+00 -1.215e+01 -1.855e+01 -2.625e+01 -3.525e+01
 -4.500e+01 -5.500e+01 -6.500e+01 -7.500e+01 -8.500e+01 -9.500e+01
 -1.050e+02 -1.150e+02 -1.250e+02 -1.350e+02 -1.465e+02 -1.615e+02
 -1.800e+02 -2.000e+02 -2.200e+02 -2.400e+02 -2.600e+02 -2.800e+02
 -3.010e+02 -3.270e+02 -3.610e+02 -4.025e+02 -4.500e+02 -5.000e+02
 -5.515e+02 -6.140e+02 -7.000e+02 -8.000e+02 -9.000e+02 -1.000e+03
 -1.100e+03 -1.225e+03 -1.400e+03 -1.600e+03 -1.800e+03 -2.010e+03
 -2

  predict_proba.1-preprocess.2-feature_THETA.4-scale_transform: 8442 ms
  predict_proba.1-preprocess.2-feature_THETA.5-reduce_fit: 0 ms
  predict_proba.1-preprocess.2-feature_THETA.6-reduce_transform: 954 ms
  predict_proba.1-preprocess.2-feature_THETA.total: 10456 ms
  predict_proba.1-preprocess: 10457 ms
  predict_proba.1-preprocess.3-homogeniser: 14 ms
[-2.100e+00 -6.700e+00 -1.215e+01 -1.855e+01 -2.625e+01 -3.525e+01
 -4.500e+01 -5.500e+01 -6.500e+01 -7.500e+01 -8.500e+01 -9.500e+01
 -1.050e+02 -1.150e+02 -1.250e+02 -1.350e+02 -1.465e+02 -1.615e+02
 -1.800e+02 -2.000e+02 -2.200e+02 -2.400e+02 -2.600e+02 -2.800e+02
 -3.010e+02 -3.270e+02 -3.610e+02 -4.025e+02 -4.500e+02 -5.000e+02
 -5.515e+02 -6.140e+02 -7.000e+02 -8.000e+02 -9.000e+02 -1.000e+03
 -1.100e+03 -1.225e+03 -1.400e+03 -1.600e+03 -1.800e+03 -2.010e+03
 -2.270e+03 -2.610e+03 -3.000e+03 -3.400e+03 -3.800e+03 -4.200e+03
 -4.600e+03 -5.000e+03 -5.400e+03 -5.800e+03]
  predict_proba.1-preprocess.2-feature_SALT.1-ravel: 1140 ms

  predict_proba.1-preprocess.2-feature_SALT.4-scale_transform: 7628 ms
  predict_proba.1-preprocess.2-feature_SALT.5-reduce_fit: 0 ms
  predict_proba.1-preprocess.2-feature_SALT.6-reduce_transform: 923 ms
  predict_proba.1-preprocess.2-feature_SALT.total: 9828 ms
  predict_proba.1-preprocess: 9829 ms
  predict_proba.1-preprocess.3-homogeniser: 14 ms
  predict_proba.1-preprocess.4-xarray: 98 ms
  predict_proba.1-preprocess: 18250 ms
  predict_proba.predict: 356 ms
  predict_proba.score: 304 ms
  predict_proba.xarray: 15696 ms
  predict_proba: 34611 ms
  predict.1-preprocess.1-mask: 173 ms
[-2.100e+00 -6.700e+00 -1.215e+01 -1.855e+01 -2.625e+01 -3.525e+01
 -4.500e+01 -5.500e+01 -6.500e+01 -7.500e+01 -8.500e+01 -9.500e+01
 -1.050e+02 -1.150e+02 -1.250e+02 -1.350e+02 -1.465e+02 -1.615e+02
 -1.800e+02 -2.000e+02 -2.200e+02 -2.400e+02 -2.600e+02 -2.800e+02
 -3.010e+02 -3.270e+02 -3.610e+02 -4.025e+02 -4.500e+02 -5.000e+02
 -5.515e+02 -6.140e+02 -7.000e+02 -8.000e+02 -9.000e+02 -1.000e+03
 -1

...

  predict_proba.1-preprocess.2-feature_SALT.4-scale_transform: 3452 ms
  predict_proba.1-preprocess.2-feature_SALT.5-reduce_fit: 0 ms
  predict_proba.1-preprocess.2-feature_SALT.6-reduce_transform: 900 ms
  predict_proba.1-preprocess.2-feature_SALT.total: 5584 ms
  predict_proba.1-preprocess: 5585 ms
  predict_proba.1-preprocess.3-homogeniser: 13 ms
  predict_proba.1-preprocess.4-xarray: 94 ms
  predict_proba.1-preprocess: 13520 ms
  predict_proba.predict: 361 ms
  predict_proba.score: 278 ms
  predict_proba.xarray: 15695 ms
  predict_proba: 29859 ms
  predict.1-preprocess.1-mask: 181 ms
[-2.100e+00 -6.700e+00 -1.215e+01 -1.855e+01 -2.625e+01 -3.525e+01
 -4.500e+01 -5.500e+01 -6.500e+01 -7.500e+01 -8.500e+01 -9.500e+01
 -1.050e+02 -1.150e+02 -1.250e+02 -1.350e+02 -1.465e+02 -1.615e+02
 -1.800e+02 -2.000e+02 -2.200e+02 -2.400e+02 -2.600e+02 -2.800e+02
 -3.010e+02 -3.270e+02 -3.610e+02 -4.025e+02 -4.500e+02 -5.000e+02
 -5.515e+02 -6.140e+02 -7.000e+02 -8.000e+02 -9.000e+02 -1.000e+03
 -1

  predict.1-preprocess.2-feature_THETA.4-scale_transform: 7107 ms
  predict.1-preprocess.2-feature_THETA.5-reduce_fit: 0 ms
  predict.1-preprocess.2-feature_THETA.6-reduce_transform: 947 ms
  predict.1-preprocess.2-feature_THETA.total: 8691 ms
  predict.1-preprocess: 8692 ms
  predict.1-preprocess.3-homogeniser: 13 ms
[-2.100e+00 -6.700e+00 -1.215e+01 -1.855e+01 -2.625e+01 -3.525e+01
 -4.500e+01 -5.500e+01 -6.500e+01 -7.500e+01 -8.500e+01 -9.500e+01
 -1.050e+02 -1.150e+02 -1.250e+02 -1.350e+02 -1.465e+02 -1.615e+02
 -1.800e+02 -2.000e+02 -2.200e+02 -2.400e+02 -2.600e+02 -2.800e+02
 -3.010e+02 -3.270e+02 -3.610e+02 -4.025e+02 -4.500e+02 -5.000e+02
 -5.515e+02 -6.140e+02 -7.000e+02 -8.000e+02 -9.000e+02 -1.000e+03
 -1.100e+03 -1.225e+03 -1.400e+03 -1.600e+03 -1.800e+03 -2.010e+03
 -2.270e+03 -2.610e+03 -3.000e+03 -3.400e+03 -3.800e+03 -4.200e+03
 -4.600e+03 -5.000e+03 -5.400e+03 -5.800e+03]
  predict.1-preprocess.2-feature_SALT.1-ravel: 1345 ms
[-2.100e+00 -6.700e+00 -1.215e+01 -1.855e+0

  predict.1-preprocess.2-feature_SALT.4-scale_transform: 4647 ms
  predict.1-preprocess.2-feature_SALT.5-reduce_fit: 0 ms
  predict.1-preprocess.2-feature_SALT.6-reduce_transform: 890 ms
  predict.1-preprocess.2-feature_SALT.total: 6901 ms
  predict.1-preprocess: 6902 ms
  predict.1-preprocess.3-homogeniser: 12 ms
  predict.1-preprocess.4-xarray: 95 ms
  predict.1-preprocess: 14132 ms
  predict.predict: 269 ms
  predict.score: 291 ms
  predict.xarray: 3488 ms
  predict: 18185 ms
  predict_proba.1-preprocess.1-mask: 425 ms
[-2.100e+00 -6.700e+00 -1.215e+01 -1.855e+01 -2.625e+01 -3.525e+01
 -4.500e+01 -5.500e+01 -6.500e+01 -7.500e+01 -8.500e+01 -9.500e+01
 -1.050e+02 -1.150e+02 -1.250e+02 -1.350e+02 -1.465e+02 -1.615e+02
 -1.800e+02 -2.000e+02 -2.200e+02 -2.400e+02 -2.600e+02 -2.800e+02
 -3.010e+02 -3.270e+02 -3.610e+02 -4.025e+02 -4.500e+02 -5.000e+02
 -5.515e+02 -6.140e+02 -7.000e+02 -8.000e+02 -9.000e+02 -1.000e+03
 -1.100e+03 -1.225e+03 -1.400e+03 -1.600e+03 -1.800e+03 -2.010e+03
 -2

  predict_proba.1-preprocess.2-feature_THETA.4-scale_transform: 5197 ms
  predict_proba.1-preprocess.2-feature_THETA.5-reduce_fit: 0 ms
  predict_proba.1-preprocess.2-feature_THETA.6-reduce_transform: 876 ms
  predict_proba.1-preprocess.2-feature_THETA.total: 7108 ms
  predict_proba.1-preprocess: 7108 ms
  predict_proba.1-preprocess.3-homogeniser: 12 ms
[-2.100e+00 -6.700e+00 -1.215e+01 -1.855e+01 -2.625e+01 -3.525e+01
 -4.500e+01 -5.500e+01 -6.500e+01 -7.500e+01 -8.500e+01 -9.500e+01
 -1.050e+02 -1.150e+02 -1.250e+02 -1.350e+02 -1.465e+02 -1.615e+02
 -1.800e+02 -2.000e+02 -2.200e+02 -2.400e+02 -2.600e+02 -2.800e+02
 -3.010e+02 -3.270e+02 -3.610e+02 -4.025e+02 -4.500e+02 -5.000e+02
 -5.515e+02 -6.140e+02 -7.000e+02 -8.000e+02 -9.000e+02 -1.000e+03
 -1.100e+03 -1.225e+03 -1.400e+03 -1.600e+03 -1.800e+03 -2.010e+03
 -2.270e+03 -2.610e+03 -3.000e+03 -3.400e+03 -3.800e+03 -4.200e+03
 -4.600e+03 -5.000e+03 -5.400e+03 -5.800e+03]
  predict_proba.1-preprocess.2-feature_SALT.1-ravel: 623 ms
[-

  predict_proba.1-preprocess.2-feature_SALT.4-scale_transform: 5661 ms
  predict_proba.1-preprocess.2-feature_SALT.5-reduce_fit: 0 ms
  predict_proba.1-preprocess.2-feature_SALT.6-reduce_transform: 864 ms
  predict_proba.1-preprocess.2-feature_SALT.total: 7382 ms
  predict_proba.1-preprocess: 7383 ms
  predict_proba.1-preprocess.3-homogeniser: 12 ms
  predict_proba.1-preprocess.4-xarray: 93 ms
  predict_proba.1-preprocess: 15536 ms
  predict_proba.predict: 398 ms
  predict_proba.score: 295 ms
  predict_proba.xarray: 15455 ms
  predict_proba: 31689 ms
  predict.1-preprocess.1-mask: 194 ms
[-2.100e+00 -6.700e+00 -1.215e+01 -1.855e+01 -2.625e+01 -3.525e+01
 -4.500e+01 -5.500e+01 -6.500e+01 -7.500e+01 -8.500e+01 -9.500e+01
 -1.050e+02 -1.150e+02 -1.250e+02 -1.350e+02 -1.465e+02 -1.615e+02
 -1.800e+02 -2.000e+02 -2.200e+02 -2.400e+02 -2.600e+02 -2.800e+02
 -3.010e+02 -3.270e+02 -3.610e+02 -4.025e+02 -4.500e+02 -5.000e+02
 -5.515e+02 -6.140e+02 -7.000e+02 -8.000e+02 -9.000e+02 -1.000e+03
 -1

In [6]:
m

<pcm 'gmm' (K: 5, F: 2)>
Number of class: 5
Number of feature: 2
Feature names: odict_keys(['THETA', 'SALT'])
Fitted: True
Feature: 'THETA'
	 Interpoler: <class 'pyxpcm.utils.Vertical_Interpolator'>
	 Scaler: 'normal', <class 'sklearn.preprocessing._data.StandardScaler'>
	 Reducer: True, <class 'sklearn.decomposition._pca.PCA'>
Feature: 'SALT'
	 Interpoler: <class 'pyxpcm.utils.Vertical_Interpolator'>
	 Scaler: 'normal', <class 'sklearn.preprocessing._data.StandardScaler'>
	 Reducer: True, <class 'sklearn.decomposition._pca.PCA'>
Classifier: 'gmm', <class 'sklearn.mixture._gaussian_mixture.GaussianMixture'>
	 log likelihood of the training set: -1.584785